# pair paragraphs

In [26]:
is_pl = True
par = {}
pars = []
for line in open('text'):
    k = 'pl' if is_pl else 'en'
    par[k] = par.get(k, [])
    if line == '\n':
        par[k] = " ".join(par[k])
        
        if not is_pl:
            # the pars go: pl, en, pl, en
            pars.append(par)
            par = {}
            
        is_pl = not is_pl
        continue
    par[k].append(line.strip())
    
import json

with open('pars', 'w') as f:
    for par in pars:
        f.write(json.dumps(par))